**14. PyTorch's NN module**

* The torch.nn module is a core library that provides a wide array of classes and functions designed to help developers build neural networks efficiently and effectively. It abstracts the complexity of creating and training neural networks by offering pre-built layers, loss functions, activation functions, and other utilities, enabling you to focus on designing and experimenting with model architectures.

Key Components from `tr.nn`:

1. Module(Layers):  
   * nn.Module: The base class for all neural network modules. Your custom models and layers should subclass the class.
   * Common Layers: includes layers like `nn.linear`(fully connected layer), `nn.Conv2d` (convolutional layer), `nn.LSTM` (recurrent layer) and many others.
2. Activation functions:
   * Functions like `nn.ReLU`, `nn.Sigmoid`, and `nn.Tanh` introduce non-linearities to the neural network, allowing to learn complex patterns.
3. Loss Functions:
   * Provides loss functions such as `nn.CrossEntropyLoss`, `nn.MSELoss`, `nn.NLLLoss` (negative log-likelihood) to quantify the difference between model's predictions and the actual targets.
4. Container Modules:
   * `nn.Sequential`: A sequential container to stack layers in order.
5. Regularization and Dropout:
   * Layers like `nn.Dropout` and `nn.BatchNorm2d` help to prevent overfitting and improve model's ability to generalize to new data.

Keynotes of `tr.optim`:
* tr.optim is a module in PyTorch that provides a variety of optimization algorithms used to update the parameters of your model during training.
* It includes common optimizers like Stochasitic Gradient Descent (SGD), Adam, RMSprop, and more.
* It handles weight updates efficiently, including additional features like learning rate scheduling and weight decay (regularization).

The `model.parameters()` method in PyTorch retrieves an iterator over all the trainable parameters (weights and biases) in a model. These parameters are instances of tr.nn.Parameter and include:
* Weights
* Biases
  
The optimizer uses these parameters to compute gradient and update them during training.

In [1]:
import numpy as np
import torch as tr
import torch.nn as nn
from torchinfo import summary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [2]:
# dataset: Breast Cancer Detection
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
# Perform train-test-split
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)

In [5]:
# Pre-processing (normalizing)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [6]:
# label encoding
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)
y_train[:5],y_test[:5]

(array([0, 0, 1, 0, 0]), array([0, 0, 1, 1, 1]))

In [7]:
# convert to PyTorch tensors
X_train_tensor = tr.from_numpy(X_train).to(tr.float32)
X_test_tensor = tr.from_numpy(X_test).to(tr.float32)

y_train_tensor = tr.from_numpy(y_train).to(tr.float32)
y_test_tensor = tr.from_numpy(y_test).to(tr.float32)

In [ ]:
X_train_tensor.shape,y_train_tensor.shape

**14.01 Basic neural network creation**

In [ ]:
# defining random seed 
random_seed = 14
tr.manual_seed(random_seed)

# defining the model

class myNN(nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.linear = nn.Linear(in_features=input_size, out_features=1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x
    
    

In [ ]:
# test the model's forward pass
test_x  = tr.randn(10,5)

model = myNN(test_x.shape[1])

# call model for forward pass
model(test_x)

In [ ]:
model.linear.weight,model.linear.bias

In [ ]:
# visualizing summary
summary(model,input_size=(10,5))

**14.02 Creating neural network with hidden layers**

In [8]:
class hNet(nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.linear1 = nn.Linear(in_features=input_size,out_features=3)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(in_features=3,out_features=1)
        self.sigmoid = nn.Sigmoid()
    

    def forward(self,x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x

In [ ]:
# test the model's forward pass
test_x  = tr.randn(10,5)

model = hNet(test_x.shape[1])

# call model for forward pass
model(test_x)

In [ ]:
summary(model,input_size=(10,5))

In [ ]:
model.parameters # object containing all layers and activations

In [ ]:
model.parameters() # iterator

In [ ]:
# using `.view()` to reshape
a = tr.randn(5,)
print(a,a.shape)
a = a.view(-1,1)
print(a,a.shape)

In [10]:
# important parameter of Neural Network
learning_rate = 0.1
epochs = 25

In [17]:
### model training
model = hNet(X_test_tensor.shape[1])

# loss function
loss_function = nn.BCELoss()

# define optimizer 
optimizer = tr.optim.SGD(model.parameters(),lr = learning_rate)

# Loop no of epochs:
for epochi in range(epochs):
    # 1. Forward pass
    y_pred = model(X_train_tensor)

    # 2. Loss Calculation
    loss = loss_function(y_pred,y_train_tensor.view(-1,1))

    # 5. zero gradients (stop accumulation of gradient) [Better strategy to clear first]
    optimizer.zero_grad()

    # 3. Backward pass
    loss.backward()


    # 4. Update parameters 
    optimizer.step() # no-need to disable gradients, it does it automatically


    # 6. printing loss in each epoch
    print(f'Epoch: {epochi+1}, Loss:{loss.item()}')

Epoch: 1, Loss:0.6553032398223877
Epoch: 2, Loss:0.639369010925293
Epoch: 3, Loss:0.6248773336410522
Epoch: 4, Loss:0.6109257340431213
Epoch: 5, Loss:0.5977701544761658
Epoch: 6, Loss:0.585182785987854
Epoch: 7, Loss:0.5727420449256897
Epoch: 8, Loss:0.5605580806732178
Epoch: 9, Loss:0.5489643216133118
Epoch: 10, Loss:0.537971019744873
Epoch: 11, Loss:0.5275160074234009
Epoch: 12, Loss:0.517610490322113
Epoch: 13, Loss:0.5078899264335632
Epoch: 14, Loss:0.4985869824886322
Epoch: 15, Loss:0.48964494466781616
Epoch: 16, Loss:0.4811352789402008
Epoch: 17, Loss:0.47298747301101685
Epoch: 18, Loss:0.4650864899158478
Epoch: 19, Loss:0.457489550113678
Epoch: 20, Loss:0.4501720368862152
Epoch: 21, Loss:0.4431031346321106
Epoch: 22, Loss:0.43622225522994995
Epoch: 23, Loss:0.42956674098968506
Epoch: 24, Loss:0.423117071390152
Epoch: 25, Loss:0.4168704152107239


In [15]:
# model evaluation
with tr.no_grad():
    y_pred = model.forward(X_test_tensor)
thresh = 0.5
# y_pred_thresh = [int(y.item()>thresh) for y in y_pred]
y_pred_thresh = [int(y>thresh) for y in y_pred]
y_pred_thresh[:5]

[0, 0, 1, 1, 1]

In [16]:
accuracy = (sum(y_pred_thresh == y_test)/len(y_test))*100
print(f'Accuaracy: {accuracy:.2f} %')

Accuaracy: 94.74 %
